In [1]:
import pandas as pd
import numpy as np
train=pd.read_csv('train_modified.csv')
train.Loan_Status.replace({'Y':1,'N':0},inplace=True)
train.head()

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,loan,Loan_Status
0,1,0,0,1,0,1.0,3,1,3,1
1,1,1,1,1,0,1.0,1,1,3,0
2,1,1,0,1,1,1.0,3,0,3,1
3,1,1,0,0,0,1.0,3,0,3,1
4,1,0,0,1,0,1.0,3,1,4,1


In [2]:
x=train.drop('Loan_Status',axis=1)
y=train.Loan_Status

In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=0)

In [4]:
from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression()
lr_clf.fit(x_train,y_train)
lr_clf.score(x_test,y_test)

0.7967479674796748

In [5]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
cv=ShuffleSplit(n_splits=5,test_size=.2,random_state=0)
score=cross_val_score(LogisticRegression(),x,y,cv=cv)
score.mean()

0.8

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
def find_best_model(x,y):
    algos={
        'logistic_regression': {
            'model': LogisticRegression(),
            'params': {
                
            }
        },
        'randomforest':{
            'model': RandomForestClassifier(),
            'params': {
                'max_leaf_nodes': [5,8,10,30,50,100]
            }
        },
        'decision_tree':{
            'model': DecisionTreeClassifier(),
            'params': {
                'max_leaf_nodes': [2,5,10,30],
            }
        },
        'svm':{
            'model':SVC(),
            'params':{
                'C':[1,10],
                'kernel':['rbf','linear','poly']
            }
        }
    }
    scores=[]
    cv=KFold(n_splits=5,random_state=0,shuffle=True)
    for algo_name,config in algos.items():
        gs=GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(x,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [7]:
find_best_model(x,y)

,model,best_score,best_params
0,logistic_regression,0.80745,{}
1,randomforest,0.80909,{'max_leaf_nodes': 5}
2,decision_tree,0.80909,{'max_leaf_nodes': 2}
3,svm,0.80909,"{'C': 1, 'kernel': 'linear'}"
